# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 2.0.10 which is incompatible.


The environment is already saved in the Workspace and can be accessed at the file path provided below. 

In [2]:
from unityagents import UnityEnvironment
import numpy as np
import time;

In [3]:
env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.         -6.65278625 -1.5        -0.          0.
  6.83172083  6.         -0.          0.        ]


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)


class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, hidden_units, action_size, seed):
        """Initialize parameters and build model.
        Params
        ======
            hidden_units (array): Number of nodes for layers
            seed (int): Random seed
            gate (function): activation function
            final_gate (function): final activation function
        """
        super(Actor, self).__init__()
        
        self.seed = torch.manual_seed(seed)
        
        self.input_layer = nn.Linear(state_size, hidden_units[0]);
        self.batchnorm_layer = nn.BatchNorm1d(hidden_units[0]);
        
        self.hidden_layer_1 = nn.Linear(hidden_units[0], hidden_units[1]);
        self.output_layer = nn.Linear(hidden_units[1], action_size);
        
        self.reset_parameters()

    def reset_parameters(self):
        self.input_layer.weight.data.uniform_(*hidden_init(self.input_layer));
        self.hidden_layer_1.weight.data.uniform_(*hidden_init(self.hidden_layer_1));
        
        self.output_layer.weight.data.uniform_(-3e-3, 3e-3);

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        
        out = self.input_layer(state);
        out = F.relu(self.batchnorm_layer(out));
        out = F.relu(self.hidden_layer_1(out));
        out = F.tanh(self.output_layer(out));
        return out;
    
class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, hidden_units, action_size, seed):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            hidden_units (array): Number of nodes for layers
            seed (int): Random seed
            gate (function): activation function
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        
        self.input_layer = nn.Linear(state_size, hidden_units[0]);
        
        self.batchnorm_layer = nn.BatchNorm1d(hidden_units[0]);
        self.hidden_layer_1 = nn.Linear(hidden_units[0] + action_size, hidden_units[1]);
        self.dropout_layer = nn.Dropout(p=0.2);
        self.output_layer = nn.Linear(hidden_units[1], action_size);
        self.reset_parameters()

    def reset_parameters(self):      
        self.input_layer.weight.data.uniform_(*hidden_init(self.input_layer));
        self.hidden_layer_1.weight.data.uniform_(*hidden_init(self.hidden_layer_1));
        
        self.output_layer.weight.data.uniform_(-3e-3, 3e-3);

    def forward(self, state, action):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        x_state = self.input_layer(state);
        x_state = F.relu(self.batchnorm_layer(x_state));

        out = torch.cat((x_state, action), dim=1);
        out = F.relu(self.hidden_layer_1(out));
        #x = F.relu(self.hidden_layer_2(x));
        out = self.dropout_layer(out);
        out = self.output_layer(out);
        
        return out;

In [38]:
import numpy as np
import random
import copy
from collections import namedtuple, deque
import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 128        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR_ACTOR = 1e-4         # learning rate of the actor
LR_CRITIC = 1e-4        # learning rate of the critic
WEIGHT_DECAY = 0        # L2 weight decay

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, replay_memory, random_seed):
        """Initialize an Agent object.
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            random_seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(random_seed)

        # Actor Network (w/ Target Network)
        self.actor_local = Actor(self.state_size, (128, 64), self.action_size, random_seed).to(device)
        self.actor_target = Actor(self.state_size, (128, 64), self.action_size, random_seed).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=LR_ACTOR)

        # Critic Network (w/ Target Network)
        self.critic_local = Critic(self.state_size, (128, 64), self.action_size, random_seed).to(device)
        self.critic_target = Critic(self.state_size, (128, 64), self.action_size, random_seed).to(device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=LR_CRITIC, weight_decay=WEIGHT_DECAY)

        self.actor_target.load_state_dict(self.actor_local.state_dict())
        self.critic_target.load_state_dict(self.critic_local.state_dict())

        # Noise process
        self.noise = OUNoise(action_size, random_seed)

        # Replay memory
        self.memory = replay_memory;


    def step(self, state, action, reward, next_state, done):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        self.memory.add(state, action, reward, next_state, done)

        # Learn, if enough samples are available in memory
        if len(self.memory) > BATCH_SIZE:
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)

    def act(self, state, add_noise=True):
        """Returns actions for given state as per current policy."""
        state = torch.from_numpy(state).float().to(device)
        #print('state {}'.format(state));
        #print('state shpae {}'.format(state.shape));
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state).cpu().data.numpy()
        self.actor_local.train()
        if add_noise:
            action += self.noise.sample()
        return np.clip(action, -1, 1)

    def reset(self):
        self.noise.reset()

    def learn(self, experiences, gamma):
        """Update policy and value parameters using given batch of experience tuples.
        Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value
        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # ---------------------------- update critic ---------------------------- #
        # Get predicted next-state actions and Q values from target models
        actions_next = self.actor_target(next_states)
        Q_targets_next = self.critic_target(next_states, actions_next)
        # Compute Q targets for current states (y_i)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        # Compute critic loss
        Q_expected = self.critic_local(states, actions)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic_local.parameters(), 1)
        self.critic_optimizer.step()

        # ---------------------------- update actor ---------------------------- #
        # Compute actor loss
        actions_pred = self.actor_local(states)
        actor_loss = -self.critic_local(states, actions_pred).mean()
        # Minimize the loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # ----------------------- update target networks ----------------------- #
        self.soft_update(self.critic_local, self.critic_target, TAU)
        self.soft_update(self.actor_local, self.actor_target, TAU)

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)


class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.1):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state


class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(
            device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(
            device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [39]:
class MultiAgents():
    def __init__(self, num_agents, state_size, action_size, random_seed):
        self.num_agents = num_agents;
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed);
        self.agents = [Agent(state_size, action_size, self.memory, random_seed) for agent_posit in range(num_agents)];
    
    def reset(self):
        [agent.reset() for agent in self.agents];
    
    def step(self, states, actions, rewards, next_states, dones):
        [self.agents[posit].step(states[posit], actions[posit], rewards[posit], next_states[posit], dones[posit]) for posit in range(self.num_agents)];
    
    def act(self, states):
        actions = [self.agents[posit].act(np.array([states[posit]])) for posit in range(self.num_agents)];
        return actions;
    
    def save(self):
        for posit in range(self.num_agents):
            torch.save(self.agents[posit].actor_local.state_dict(), 'checkpoint_actor_%d.pth'%(posit));
            torch.save(self.agents[posit].actor_local.state_dict(), 'checkpoint_critic_%d.pth'%(posit));

3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [40]:
for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291


In [41]:
NUM_STATES = env_info.vector_observations.shape[1];
NUM_ACTIONS = brain.vector_action_space_size;
NUM_AGENTS = len(env_info.agents)

NUM_STATES, NUM_ACTIONS, NUM_AGENTS

(24, 2, 2)

In [42]:
agent = MultiAgents(num_agents=2, state_size=NUM_STATES, action_size=NUM_ACTIONS, random_seed=10)

In [43]:
def multi_ddpg(n_episodes=5000, max_t=2000):
    init_time = time.time();
    scores_deque = deque(maxlen=100);
    scores = []
    max_score = -np.Inf;
    
    for i_episode in range(1, n_episodes+1):
        
        ep_init_time = time.time();
        env_info = env.reset(train_mode=True)[brain_name];
        states = env_info.vector_observations;
        
        agent.reset();
        agent_scores = np.zeros(num_agents);
        
        for t in range(max_t):
            actions = agent.act(states);
            next_env_info = env.step(actions)[brain_name];
            
            next_states, rewards, dones = next_env_info.vector_observations, next_env_info.rewards, next_env_info.local_done;
    
            agent.step(states, actions, rewards, next_states, dones);
            
            agent_scores = agent_scores + rewards;
            
            states = next_states;
            
        best_score = np.max(agent_scores);
        scores_deque.append(best_score);
        scores.append(best_score);
        
        print('\rEpisode %d\t Done in %.2f/%.2f\t Scores: %s\tAvg Score %.2f'%(i_episode, (time.time() - ep_init_time), (time.time() - init_time), ['%.2f'%sc for sc in agent_scores], np.mean(scores_deque)))
        
        if i_episode % 100 == 0:
            agent.save();
            print('\nEpisode %d\t Done in %.2f/%.2f\t Scores: %s\tAvg Score %.2f'%(i_episode, (time.time() - ep_init_time), (time.time() - init_time), ['%.2f'%sc for sc in agent_scores], np.mean(scores_deque)))

        if np.mean(scores_deque)>=0.5:
            agent.save();
            print('\nEnvironment Solved: %d\t Done in %.2f/%.2f\t Scores: %s\tAvg Score %.2f'%(i_episode, (time.time() - ep_init_time), (time.time() - init_time), ['%.2f'%sc for sc in agent_scores], np.mean(scores_deque)))
            break;
            
    return scores;

In [44]:
print('Starting to');
multi_ddpg()

Starting to
Episode 1	 Done in 61.93/61.93	 Scores: ['-0.52', '-0.53']	Avg Score -0.52
Episode 2	 Done in 66.46/128.38	 Scores: ['0.81', '0.14']	Avg Score 0.15
Episode 3	 Done in 66.40/194.79	 Scores: ['1.00', '-0.60']	Avg Score 0.43
Episode 4	 Done in 66.97/261.76	 Scores: ['-0.53', '-0.07']	Avg Score 0.31
Episode 5	 Done in 66.88/328.64	 Scores: ['0.93', '1.53']	Avg Score 0.55

Environment Solved: 5	 Done in 66.88/328.65	 Scores: ['0.93', '1.53']	Avg Score 0.55


[-0.51999998092651367,
 0.81000003032386303,
 1.0000000335276127,
 -0.069999976083636284,
 1.5300000477582216]

When finished, you can close the environment.

In [8]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 